In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch import nn, optim
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load datasets
train_df = pd.read_csv('/content/drive/Shareddrives/Proyecto de Administracion/NUEVO/Train.csv')
test_df = pd.read_csv('/content/drive/Shareddrives/Proyecto de Administracion/NUEVO/Testing.csv')


In [ ]:
test_df.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,smean,dmean,trans_depth,response_body_len,label
0,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,254,43,43,0,0,0
1,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,252,52,1106,0,0,0
2,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,252,46,824,0,0,0
3,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,52,64,0,0,0
4,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,252,53,45,0,0,0


In [ ]:
# Extract features and labels
features = ['dur', 'proto', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl',
            'smean', 'dmean', 'service', 'state']
target = 'label'

# Encode categorical features
proto_encoder = LabelEncoder()
service_encoder = LabelEncoder()
state_encoder = LabelEncoder()

train_df['proto'] = proto_encoder.fit_transform(train_df['proto'])
train_df['service'] = service_encoder.fit_transform(train_df['service'])
train_df['state'] = state_encoder.fit_transform(train_df['state'])

test_df['proto'] = proto_encoder.transform(test_df['proto'])
test_df['service'] = service_encoder.transform(test_df['service'])
test_df['state'] = state_encoder.transform(test_df['state'])

# Separate features and target
X_train = train_df[features].values
y_train = train_df[target].values
X_test = test_df[features].values
y_test = test_df[target].values

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
# Save mean and scale values
scaler_mean = scaler.mean_
scaler_scale = scaler.scale_

print("Scaler mean:", scaler_mean)
print("Scaler scale:", scaler_scale)

Scaler mean: [1.35938869e+00 1.09606675e+02 2.02986637e+01 1.89695907e+01
 8.84484384e+03 1.49289186e+04 9.54061871e+04 1.79546997e+02
 7.96095665e+01 1.36751769e+02 1.24173382e+02 1.61891971e+00
 2.35517648e+00]
Scaler scale: [6.48023038e+00 2.23525372e+01 1.36887207e+02 1.10257956e+02
 1.74765146e+05 1.43653808e+05 1.65400507e+05 1.02939718e+02
 1.10506548e+02 2.04676776e+02 2.58316319e+02 2.30514410e+00
 8.67939407e-01]


In [ ]:
test_df.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,smean,dmean,trans_depth,response_body_len,label
0,0.121478,113,0,2,6,4,258,172,74.087490,252,254,43,43,0,0,0
1,0.649902,113,0,2,14,38,734,42014,78.473372,62,252,52,1106,0,0,0
2,1.623129,113,0,2,8,16,364,13186,14.170161,62,252,46,824,0,0,0
3,1.681642,113,3,2,12,12,628,770,13.677108,62,252,52,64,0,0,0
4,0.449454,113,0,2,10,6,534,268,33.373826,254,252,53,45,0,0,0


In [ ]:
print("proto_encoder.classes_", proto_encoder.classes_)
print("service_encoder.classes_", service_encoder.classes_)
print("state_encoder.classes_", state_encoder.classes_)

proto_encoder.classes_ ['3pc' 'a/n' 'aes-sp3-d' 'any' 'argus' 'aris' 'arp' 'ax.25' 'bbn-rcc'
 'bna' 'br-sat-mon' 'cbt' 'cftp' 'chaos' 'compaq-peer' 'cphb' 'cpnx'
 'crtp' 'crudp' 'dcn' 'ddp' 'ddx' 'dgp' 'egp' 'eigrp' 'emcon' 'encap'
 'etherip' 'fc' 'fire' 'ggp' 'gmtp' 'gre' 'hmp' 'i-nlsp' 'iatp' 'ib'
 'icmp' 'idpr' 'idpr-cmtp' 'idrp' 'ifmp' 'igmp' 'igp' 'il' 'ip' 'ipcomp'
 'ipcv' 'ipip' 'iplt' 'ipnip' 'ippc' 'ipv6' 'ipv6-frag' 'ipv6-no'
 'ipv6-opts' 'ipv6-route' 'ipx-n-ip' 'irtp' 'isis' 'iso-ip' 'iso-tp4'
 'kryptolan' 'l2tp' 'larp' 'leaf-1' 'leaf-2' 'merit-inp' 'mfe-nsp' 'mhrp'
 'micp' 'mobile' 'mtp' 'mux' 'narp' 'netblt' 'nsfnet-igp' 'nvp' 'ospf'
 'pgm' 'pim' 'pipe' 'pnni' 'pri-enc' 'prm' 'ptp' 'pup' 'pvp' 'qnx' 'rdp'
 'rsvp' 'rtp' 'rvd' 'sat-expak' 'sat-mon' 'sccopmce' 'scps' 'sctp' 'sdrp'
 'secure-vmtp' 'sep' 'skip' 'sm' 'smp' 'snp' 'sprite-rpc' 'sps' 'srp'
 'st2' 'stp' 'sun-nd' 'swipe' 'tcf' 'tcp' 'tlsp' 'tp++' 'trunk-1'
 'trunk-2' 'ttp' 'udp' 'unas' 'uti' 'vines' 'visa' 'vmtp' 'vrr

In [ ]:
# Define the model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(len(features), 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.sigmoid(self.fc4(x))
        return x

model = Net()

In [ ]:
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Training loop
epochs = 50

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Calculate accuracy
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    scheduler.step()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}, Accuracy: {accuracy}%')


Epoch 1/50, Loss: 0.15478113878885433, Accuracy: 93.21493546860118%
Epoch 2/50, Loss: 0.14296134874415006, Accuracy: 93.38888223518744%
Epoch 3/50, Loss: 0.13951349910756533, Accuracy: 93.41397619495726%
Epoch 4/50, Loss: 0.1381844264332776, Accuracy: 93.40371048414232%
Epoch 5/50, Loss: 0.13716655813062387, Accuracy: 93.40085889780485%
Epoch 6/50, Loss: 0.1360974419304598, Accuracy: 93.43450761658711%
Epoch 7/50, Loss: 0.1356165640158103, Accuracy: 93.4367888856571%
Epoch 8/50, Loss: 0.13476463720868648, Accuracy: 93.4350779338546%
Epoch 9/50, Loss: 0.13446017174580455, Accuracy: 93.44135142379706%
Epoch 10/50, Loss: 0.13422646086635817, Accuracy: 93.44705459647201%
Epoch 11/50, Loss: 0.13111012424920163, Accuracy: 93.48355490159176%
Epoch 12/50, Loss: 0.13007021493339627, Accuracy: 93.48640648792923%
Epoch 13/50, Loss: 0.12959256235537303, Accuracy: 93.4921096606042%
Epoch 14/50, Loss: 0.1296264549955236, Accuracy: 93.48868775699921%
Epoch 15/50, Loss: 0.12948427096104426, Accuracy: 

In [ ]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


Accuracy: 0.9346131252815941
Precision: 0.9123952535323261
Recall: 0.9999413445504898
F1 Score: 0.9541643845474207


In [ ]:
# Save the model
torch.save(model.state_dict(), 'model_final.pth')
print("Model saved to model.pth")


Model saved to model.pth
